<a href="https://colab.research.google.com/github/Progc-unlam/material-progc/blob/main/Trabajo_Pr%C3%A1ctico_N%C2%B0_3_OpenMP_MPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programación Concurrente - TP3 - Parte 1

Utilizamos 2 prácticas con OpenMP desarrolladas en lenguaje C/C++ nativo en plataforma colab.

# **OpenMP**

-----
## Ejercicio 1 - Código Faltante
Suponiendo que una empresa desea contabilizar las ventas totales de todas sus sucursales en un periodo de 15 días, se solicita completar la sección de código faltante con el objetivo de realizar dicha tarea de forma concurrente.

Nota: El número de sucursales es un parámetro de entrada.

In [ ]:
%%writefile total_ventas.cpp

#include <omp.h>
#include <stdio.h>
#include <stdlib.h>

#define SEED 4
#define LIMIT_INF 0
#define LIMIT_SUP 100
#define DAYS 15


int main(int argc, char* argv[]){
    if(argv[1])
    {
        int count_suc = atoi( argv[1] );
        int ventas_x_sucursal[count_suc][DAYS];
        int total = 0;

        srand(SEED);
        for (int i = 0; i < count_suc; i++)
        {
            for (int j = 0; j < DAYS; j++)
            {
                ventas_x_sucursal[i][j] = (rand() % (LIMIT_SUP - LIMIT_INF + 1)) + LIMIT_INF;
            }
        }

        for (int i = 0; i < count_suc; i++)
        {
            for (int j = 0; j < DAYS; j++)
            {
                printf("%d \t", ventas_x_sucursal[i][j]);
            }
            printf("\n");
        }
        // Complete el código aquí //

        /////////////////////////////
        printf("%d\n", total);
    }
    else
    {
        printf("Por favor, ingrese la cantidad de sucursales");
    }
}

Writing total_ventas.cpp


In [ ]:
!g++ -o total_ventas -fopenmp total_ventas.cpp

In [ ]:
%env OMP_NUM_THREADS=5
!./total_ventas 5

env: OMP_NUM_THREADS=5
78 	30 	7 	32 	71 	10 	85 	11 	85 	26 	11 	96 	70 	80 	21 	
52 	85 	65 	62 	20 	59 	97 	83 	82 	47 	78 	98 	67 	2 	39 	
47 	46 	70 	20 	78 	6 	31 	62 	18 	15 	54 	96 	10 	23 	76 	
98 	42 	26 	62 	3 	46 	20 	100 	95 	69 	12 	73 	32 	79 	75 	
72 	93 	87 	7 	12 	65 	13 	9 	93 	98 	24 	47 	94 	1 	36 	
0


---
## Ejercicio 2 - Axpy

La función Axpy es parte de las bibliotecas BLAS [3]. Ella se encarga de realizar la suma de vectores, con la siguiente ecuación:

<center>$Y = \alpha X + Y$</center>

En donde:
> $\alpha$: Es un escala.

> $X$: Es el vector origen.

> $Y$: Es el vector destino.

### Responda las siguientes preguntas:
a) Ejecute el siguiente código:

    - Con la variable cantidad_N para: 50, 500.000.
    - Con los valores hilos: 1, 2, 8.
    - Con 10 ciclos.

b) Repita la prueba (a), con 100 ciclos. ¿Cuál fue la diferencia?

c) ¿Cómo varía el SpeedUp a medida que aumenta o disminuye la cantidad de hilos?

d) ¿Qué sucede con la eficiencia a medida que aumentan la cantidad_N?, ¿Porqué no llega al valor ideal?.

In [ ]:
%%writefile code_axpy.cpp
// Axpy con OpenMP, usando c, ejecutado en Colab.

#include <iostream>
#include <vector>
#include <cstdlib>
#include <sys/time.h>
#include <omp.h>    // Cabecera OpenMP

// ----------------------------------------------------------------------------
// Macros que miden el tiempo.

static double dHashTiempoHistory[3];
static struct timeval tv;

#define TIEMPO_INI( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = tv.tv_sec + tv.tv_usec/1000000.0;


#define TIEMPO_FIN( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = ((tv.tv_sec + tv.tv_usec/1000000.0) - dHashTiempoHistory[ h ]) * 1000; // Devuelvo en milisegundos
#define TIEMPO_GET( h ) dHashTiempoHistory[ h ]

#define HTH_TOTAL         1
#define HTH_AXPY_SEC      2
#define HTH_AXPY_OMP      3

using namespace std;

// ----------------------------------------------------------------------------

void axpy( double alfa, vector<double> &X, vector<double> &Y )
{
  int i;

  #pragma omp parallel for
  for(i=0;i<Y.size();i++)
  {
    Y[i] = alfa * X[i] + Y[i];
  }
}

// ----------------------------------------------------------------------------

int main(int argc, char* argv[])
{
  int i,c;
  TIEMPO_INI( HTH_TOTAL )

  // Leo los parametros.
  if( argc != 4 )
  {
      std::cerr<< " Error en los parametros de indicar: (alfa), (Tamanio vector), (ciclos ejecucion)."<<argc<<std::endl;
      exit( -1 );
  }

  float alfa     = atof( argv[1] );
  int cantidad_N = atoi( argv[2] );
  int ciclos     = atoi( argv[3] );

  // --------------------------------------------
  // Defino la memoria de los vectores.

  vector<double> X( cantidad_N );
  vector<double> Y( cantidad_N );

  for (int i=0;i<X.size();i++)
  {
    X[i] = (rand()/(double)RAND_MAX)*0.73;
    Y[i] = (rand()/(double)RAND_MAX)*0.71;
  }

  // --------------------------------------------
  // Realizo la función Axpy con OpenMP.

  TIEMPO_INI( HTH_AXPY_OMP )

  for(c=0;c<ciclos;c++)
  {
    axpy( alfa, X, Y );
  }

  TIEMPO_FIN( HTH_AXPY_OMP )

  // --------------------------------------------
  // Realizo la función Axpy en forma secuencial.

  omp_set_num_threads(1);

  TIEMPO_INI( HTH_AXPY_SEC )

  for(c=0;c<ciclos;c++)
  {
    axpy( alfa, X, Y );
  }

  TIEMPO_FIN( HTH_AXPY_SEC )

  TIEMPO_FIN( HTH_TOTAL )

 std::cout<<"Valores Reales  :" <<std::endl;
 std::cout<<"Tiempo TOTAL     : "<<TIEMPO_GET(HTH_TOTAL   )<<" [ms]"<<std::endl;
 std::cout<<"Tiempo axpy Sec  : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" [ms]"<<std::endl;
 std::cout<<"Tiempo axpy Omp  : "<<TIEMPO_GET(HTH_AXPY_OMP)<<" [ms]"<<std::endl;
 std::cout<<std::endl;
 std::cout<<"SpeedUp          : (tiempo Secuencial/tiempo paralelo) : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" / "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
 std::cout<<"Eficiencia       : SpeedUp/nro procesadores            : "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))<<std::endl;
 std::cout<<"Coste Sec        : nro procesadores*Tiempo Sec         : "<<1<<" * "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;
 std::cout<<"Coste Omp        : nro procesadores*Tiempo Paralelo    : "<<omp_get_num_procs()<<" * "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
 std::cout<<"Funcion Overhead : Coste Omp - tiempo Secuencial       : "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<" - "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))-TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;


 std::cout<<std::endl;
 std::cout<<"-------------------------------------------------------------------------------------"<<std::endl;
 std::cout<<"VALORES INDEALES:"<<std::endl;
 std::cout<<"-------------------------------------------------------------------------------------"<<std::endl;
 TIEMPO_GET(HTH_AXPY_OMP) = TIEMPO_GET(HTH_AXPY_SEC) / omp_get_num_procs();
 std::cout<<"Tiempo axpy Sec  : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" [ms]"<<std::endl;
 std::cout<<"Tiempo axpy Omp  : "<<TIEMPO_GET(HTH_AXPY_OMP)<<" [ms]"<<std::endl;

 std::cout<<"SpeedUp          : (tiempo Secuencial/tiempo paralelo) : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" / "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
 std::cout<<"Eficiencia       : SpeedUp/nro procesadores            : "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))<<std::endl;
 std::cout<<"Coste Sec        : nro procesadores*Tiempo Sec         : "<<1<<" * "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;
 std::cout<<"Coste Omp        : nro procesadores*Tiempo Paralelo    : "<<omp_get_num_procs()<<" * "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
 std::cout<<"Funcion Overhead : Coste Omp - tiempo Secuencial       : "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<" - "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))-TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;


}
// ----------------------------------------------------------------------------

Overwriting code_axpy.cpp


In [ ]:
!g++ -o axpy -fopenmp code_axpy.cpp

In [ ]:
# --------------------------------------------
#@title Parámetros de ejecución { vertical-output: true }

Cantidad_N = 500000#@param {type: "number"}
Alfa       = 1.0#@param {type: "number"}
Ciclos     = 100#@param {type: "slider", min: 0, max: 100, step: 10}
Hilos      = "8"#@param [1, 2, 3, 5, 8]

# --------------------------------------------

%env OMP_NUM_THREADS=$Hilos
!./axpy $Alfa $Cantidad_N $Ciclos

env: OMP_NUM_THREADS=8
Valores Reales  :
Tiempo TOTAL     : 1100.77 [ms]
Tiempo axpy Sec  : 509.952 [ms]
Tiempo axpy Omp  : 552.19 [ms]

SpeedUp          : (tiempo Secuencial/tiempo paralelo) : 509.952 / 552.19 = 0.923508
Eficiencia       : SpeedUp/nro procesadores            : 0.923508 / 2 = 0.461754
Coste Sec        : nro procesadores*Tiempo Sec         : 1 * 509.952 = 509.952
Coste Omp        : nro procesadores*Tiempo Paralelo    : 2 * 552.19 = 1104.38
Funcion Overhead : Coste Omp - tiempo Secuencial       : 1104.38 - 509.952 = 594.428

-------------------------------------------------------------------------------------
VALORES INDEALES:
-------------------------------------------------------------------------------------
Tiempo axpy Sec  : 509.952 [ms]
Tiempo axpy Omp  : 254.976 [ms]
SpeedUp          : (tiempo Secuencial/tiempo paralelo) : 509.952 / 254.976 = 2
Eficiencia       : SpeedUp/nro procesadores            : 2 / 2 = 1
Coste Sec        : nro procesadores*Tiempo Sec        

---

# Medidas de prestaciones en algoritmos paralelos
Las técnicas de HPC buscan reducir los tiempos de ejecución, el tiempo como medida, no alcanza. Dos algoritmos pueden ejecutar en el mismo tiempo, pero uno de ellos usa menos procesadores [6,7].


## SpeedUp
Referencia a la ganancia de velocidad que se consigue con un algoritmo paralelo, al resolver el mismo problema con respecto al algoritmo secuencial.

<center>$ SpeedUp = \frac{Tiempo\_Secuencial}{Tiempo\_Paralelo} $</center>



##Eficiencia
La eficiencia normaliza el valor del SpeedUp, al dividirlo por la cantidad de procesadores que se utilizaron para alcanzar la ganancia en velocidad. Dando la idea de la porción de tiempo que los procesadores se dedican al trabajo útil.

<center>$ Eficiencia = \frac{SpeedUp}{Nro\_Procesadores} $</center>


## Coste
El coste de un algoritmo paralelo representa el tiempo realizado por todo el sistema en la resolución del problema.

<center>$ coste = Nro\_Procesadores \times Tiempo\_Algoritmo$</center>


## Función Overhead
Es la diferencia entre el Coste y el tiempo secuencial. Mientras mayor es la función overhead, peor es el comportamiento del algoritmo paralelo.

<center>$ Overhead = {Coste}-{Tiempo\_Secuencial} $</center>

---

# **MPI**

---
## Ejercicio 3 - Contar palabras

### 3.1 Armado del ambiente

Es en donde se instalar, por única vez, el módulo MPI4py de Python en el cuaderno.

In [ ]:
!pip install mpi4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.6-cp310-cp310-linux_x86_64.whl size=2746316 sha256=fc956e7a2a84cd52b63a3739e3dacc1ddff0b020c36aa8aecb30663153820439
  Stored in directory: /root/.cache/pip/wheels/4c/ca/89/8fc1fb1c620afca13bb41c630b1f948bbf446e0aaa4b762e10
Successfully built mpi4py


Desarrollar un programa que permita obtener el valor máximo de un conjunto dado de forma distribuida.

Las condiciones a tener en cuenta son:


*   Debe trabajar con al menos, 4 procesos.
*   El resultado final debe ser informado en **cada proceso**.
*   Implementar comunicación por Buffer

Salida esperada:
```
Máximo esperado:  4.0007135387559405
Máximo total:  4.0007135387559405
Proceso 0: 4.0007135387559405
Proceso 5: 4.0007135387559405
Proceso 1: 4.0007135387559405
Proceso 3: 4.0007135387559405
Proceso 2: 4.0007135387559405
Proceso 4: 4.0007135387559405
```



In [ ]:
%%writefile mpi_tp4.py

from mpi4py import MPI
import numpy as np
from IPython.display import display
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

count = 5000
sendbuf = None

if rank == 0:
  sendbuf = np.random.randn(size, count)
  display("Máximo esperado: ", sendbuf.max())

###### Inicio de código ######

##############################

Overwriting mpi_tp4.py


In [ ]:
# --------------------------------------------
# Formulario
NRO =   6#@param {type: "slider", min: 4, max: 10}
# --------------------------------------------

! mpirun --oversubscribe --allow-run-as-root -np $NRO python mpi_tp4.py

Máximo esperado:  4.01561525378836


---
# Bibliografía

[1] MARKDOWN SYNTAX Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[2] Introducción a Python: [Página Colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb)

[3] Función Axpy de biblioteca BLAS: [Referencia](https://software.intel.com/content/www/us/en/develop/documentation/mkl-developer-reference-c/top/blas-and-sparse-blas-routines/blas-routines/blas-level-1-routines-and-functions/cblas-axpy.html)

[4] Biblioteca BLAS: [Referencia](http://www.netlib.org/blas/)

[5] Tutorial Point Colab: [PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[6] F. Almeida, D. Gimenéz, A. Vidal - Introducción a la programación paralela - 2008 - Editorial Parafino.

[7] D. Jiménez-González - Introducción a las arquitecturas paralelas. [PDF](http://so-unlam.com.ar/material-clase/HPC/Arquitecturas_de_computadores_avanzadas_(Modulo_1).pdf)

[8] Github OMP Parte 1 [Link](https://github.com/MicrosoftDocs/cpp-docs/blob/main/docs/parallel/openmp/reference/openmp-directives.md#atomic)

[9] Github OMP Parte 2 [Link](https://github.com/MicrosoftDocs/cpp-docs/blob/main/docs/parallel/openmp/reference/openmp-clauses.md#nowait)

[10] Funciones OMP [Link](https://learn.microsoft.com/en-us/cpp/parallel/openmp/reference/openmp-functions?view=msvc-170)

[11] Directivas OMP [Link](https://learn.microsoft.com/es-es/cpp/parallel/openmp/reference/openmp-directives?view=msvc-170)